Question-2:

Analysis-2: 

Fetching the expertise and non-expertise section of each Reporter based on the number of articles he has written in each section

In [14]:
#ANALYSIS-2:

#Counting the number of articles in each section of a reporter using the excel prepared for analysis of data

import xlrd
import datetime

current_row = 0
sheet_num = 0

reporter_section_dictionary={} #Dictionary to store reporter-section-count of articles

# path to the file you want to extract data from
src = r'Archive_API_Data_For_Analysis.xlsx'  #Excel we prepared during analysis-1

book = xlrd.open_workbook(src)

# select the sheet that the data resids in
work_sheet = book.sheet_by_index(sheet_num)

# get the total number of rows
num_rows = work_sheet.nrows
print('Total number of rows:',num_rows-1)

for i in range(num_rows):
    if i==0: #to skip the header
        continue
        
    reporter = work_sheet.cell_value(i, 6) #Reporter name
    section = work_sheet.cell_value(i, 2) #Section name
    
    present=0 #Variable to check whether section is present in the list of sections of dictionary
    
    if len(reporter)>0:
        if reporter in reporter_section_dictionary: #If reporter is present in dictionary
            length=len(reporter_section_dictionary[reporter]) #Fetch the length of the list
            if length==0: #If list is empty append a dictionary
                reporter_section_dictionary[reporter].append({section:1})
            else: #If list is not empty
                for i in range(length):
                    if section in reporter_section_dictionary[reporter][i]: #Check for a section name in all entries of the list
                        reporter_section_dictionary[reporter][i][section]+=1 #If present, increment the number
                        present=1
                if present==0:
                    reporter_section_dictionary[reporter].append({section:1}) #If not present, append a dictionary
        else:
            reporter_section_dictionary[reporter]=[] #If reporter is not present in dictionary, add him to dictionary
            length=len(reporter_section_dictionary[reporter])
            if length==0:
                reporter_section_dictionary[reporter].append({section:1})

    i+=1
    
    
print('Length of dictionary i.e. Number of unique reporters :'+str(len(reporter_section_dictionary)))


Total number of rows: 61928
Length of dictionary i.e. Number of unique reporters :5888


In [15]:
# Writing the output of analysis i.e. section-wise count of articles for each reporter

#Writing the output of the analysis to excel

import xlsxwriter
import os

cwd = os.getcwd() 
path=cwd+'/ana_2'

# Create an excel workbook if not present and add a worksheet.
if not os.path.exists(path+'/'+'Analysis_2_Ouput_Reporterwise_Sectionwise_Data.xlsx'):
    workbook = xlsxwriter.Workbook(path+'/'+'Analysis_2_Ouput_Reporterwise_Sectionwise_Data.xlsx')

    #Adding worksheet to the workbook
    worksheet = workbook.add_worksheet('Reporterwise_Sectionwise_Data')

    #Adding headers(title) to the columns
    worksheet.write(0, 0, 'Reporter')
    worksheet.write(0, 1, 'Section')
    worksheet.write(0, 2, 'Total Number of Articles')

    # Setting rows and columns indexes
    row = 1
    col = 0

    dict_length=len(reporter_section_dictionary)

    for key in reporter_section_dictionary.keys():
        worksheet.write(row, col, key)#Reporter name
    
        section_length=len(reporter_section_dictionary[key])
        for i in range(section_length):
            for sec_key,value in reporter_section_dictionary[key][i].items():
                worksheet.write(row, col+1, sec_key) #Section name
                worksheet.write(row, col+2, value) #Count of articles

            row+=1
    workbook.close()
    print('Output Data is written to file.')
    
else:
    print('The file already exists.')

Output Data is written to file.


Expertise(Section with most number of articles) and Non-Expertise(Section with least number of articles) sections of each Reporter

In [28]:
#Expertise(Section with most number of articles)and Non-Expertise(Section with least number of articles)sections of each Reporter

#Writing Expertise and Non-Expertise sections of each month to excel

import xlsxwriter
import os
import xlrd, xlwt
from xlutils.copy import copy as xl_copy

cwd = os.getcwd() #To access the path of current working directory
path=cwd+'/ana_2'

# Create an excel workbook if not present and add a worksheet.
if not os.path.exists(path+'/'+'Analysis_2_Ouput_Reporterwise_Expertise_Non-Expertise_Section_Data.xlsx'):
    workbook = xlsxwriter.Workbook(path+'/'+'Analysis_2_Ouput_Reporterwise_Expertise_Non-Expertise_Section_Data.xlsx')
    
    #Adding worksheet to the workbook
    worksheet = workbook.add_worksheet('Expert_Non-Expert_Section_Data')
    
    #Adding headers(title) to the columns
    worksheet.write(0, 0, 'Reporter')
    worksheet.write(0, 1, 'Expertise Section with most number of articles')
    worksheet.write(0, 2, 'Non-Expertise section with least number of articles')

    # Setting rows and columns indexes
    row = 1
    col = 0

    for key in reporter_section_dictionary.keys():
        new_dict={} #Dictionary to store the values of each reporter
        cnt_list=[]
        section_length=len(reporter_section_dictionary[key])
        worksheet.write(row,col,key) #Reporter name
        for i in range(section_length):
            for sec_key,value in reporter_section_dictionary[key][i].items():
                if sec_key not in new_dict.keys():
                    new_dict[sec_key]=value
                    cnt_list.append(value) #List fetching only the article counts
                    
        if all(cnt_list[0] == item for item in cnt_list)==False: #If list  contains different entries i.e. 
                                                            #if there are sections with unique article count
                    
            #Expertise Section
            #Descending order
            from operator import itemgetter
            desc_sort_dict=sorted(new_dict.items(), key=itemgetter(1),reverse=True) 
            worksheet.write(row,col+1,desc_sort_dict[0][0]+'('+str(desc_sort_dict[0][1])+')') #Expertise section name(number of articles)
            
            #Non-Expertise Section
            #Ascending order
            from operator import itemgetter
            asc_sort_dict=sorted(new_dict.items(), key=itemgetter(1),reverse=False) 
            worksheet.write(row,col+2,asc_sort_dict[0][0]+'('+str(asc_sort_dict[0][1])+')') #Non-Expertise section name(number of articles)
    
        else: #If there are sections with same article count, print both sections as his Expertise_Setion
            value_to_be_entered="" #Variable that stores all sections with same article count
            #Descending order
            from operator import itemgetter
            desc_sort_dict=sorted(new_dict.items(), key=itemgetter(1),reverse=True) 
            for i in range(len(desc_sort_dict)):
                value_to_be_entered+=desc_sort_dict[i][0]+'('+str(desc_sort_dict[i][1])+')'
                if i<len(desc_sort_dict)-1:
                    value_to_be_entered+=' , '

            worksheet.write(row,col+1,value_to_be_entered) #Top section name(number of articles)
    
            
        row+=1
    
    workbook.close()
    print('Output Data is written to file.')
    
else:
    print('The file already exists.')


Output Data is written to file.
